In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv('../terraforming-mars-stats.csv')
unique_sorted_game_ids = pd.Series([x for _,x in sorted(zip(df.date, df.game_id), reverse=True)]).unique() # for time-based stuff

# Summary

In [4]:
df.groupby('player').\
agg(wins = ('is_winner','sum'), games = ('is_winner', 'count'), win_rate = ('is_winner','mean')).\
sort_values(by='win_rate', ascending=False)

,wins,games,win_rate
player,,,
Meister,1,1,1.000000
Pat,26,71,0.366197
Ezra,38,105,0.361905
Ben,52,147,0.353741
Matt,36,123,0.292683
Tony,31,107,0.289720
Katie,1,4,0.250000
Annie,0,1,0.000000
Hank,0,1,0.000000


# Individual Player Summary

In [76]:
player = 'Ezra'
player_df = df[df.player == player]

In [77]:
# player history
player_df.groupby('player').\
agg(wins = ('is_winner','sum'), games = ('is_winner', 'count'), win_rate = ('is_winner','mean'))

,wins,games,win_rate
player,,,
Ezra,38,105,0.361905


In [78]:
# most recent win
player_most_recent_win_game_id = player_df[player_df['is_winner']==1].sort_values(by='date', ascending=False).head(1)['game_id'].values[0]
player_most_recent_win_df = df[df.game_id == player_most_recent_win_game_id]

In [86]:
fig = make_subplots(rows=1, cols=4, subplot_titles=[f'{i}-player' for i in sorted(df.num_players.unique())])

for i, n_players in enumerate(sorted(df.num_players.unique())):
    player_df_num_players= player_df[player_df.num_players==n_players]
    other_player_df_num_players = df[(df.player!= player) & (df.num_players==n_players)]
    
    hist_data = [player_df_num_players.card_points, other_player_df_num_players.card_points]

    group_labels = [player, 'Field']
    colors = ['blue', 'grey']

    data_ = ff.create_distplot(hist_data, group_labels, colors=colors, bin_size=5)
    
    if i > 0:
        fig.add_trace(go.Histogram(data_['data'][0], marker_color='blue', showlegend=False), row=1, col=i+1)
        fig.add_trace(go.Histogram(data_['data'][1], marker_color='grey', showlegend=False), row=1, col=i+1) 
    else:
        fig.add_trace(go.Histogram(data_['data'][0], marker_color='blue'), row=1, col=i+1)
        fig.add_trace(go.Histogram(data_['data'][1], marker_color='grey'), row=1, col=i+1)

#     fig.add_trace(go.Scatter(data_['data'][2], line=dict(color='blue', width=0.5)),
#                   row=1, col=i+1)
#     fig.add_trace(go.Scatter(data_['data'][3], line=dict(color='grey', width=0.5)),
#                   row=1, col=i+1)
    
    
fig.update_layout(title_text=f'Points from Cards: {player} vs. Field', title_x=0.5, plot_bgcolor='rgba(0,0,0,0)')
fig.update_yaxes(showticklabels=False)

# Highlighted Trends

# Player ELO

In [ ]:
# https://towardsdatascience.com/developing-a-generalized-elo-rating-system-for-multiplayer-games-b9b495e87802

# scoring functions
def linear_score(p, n):
    '''
    Args:
        p : int 
            Finishing place
        n : int 
            # of players
    '''
    return (n-p)/ (n*(n-1)/2)

def exp_score(p, n, alpha=2.):
    '''
    Args:
        p : int 
            Finishing place
        n : int 
            # of players
        alpha : exists in (1,inf)'''
    return (alpha**(n-p) - 1) / np.sum([(alpha**(n-i) - 1) for i in range(1,n)])

# expected score
def expected_score(s, player, n, d=400):
    '''
    Args:
        s : dict
            Players scores, ex.: {'Tony': 90, 'Pat': 80, ...}
        player : str
            Player of interest
        n : int 
            # of players
    '''
    return np.sum([(1/(1+10**((s[i] - s[player])/d))) if i != player else 0 for i in s.keys()])/(n*(n-1)/2)

def update_rating(current_rating, expected_score, actual_score, n, k=32):
    '''
    Args:
        current_rating : int
        expected_score : float
        actual_score : float
        n : int
            # of players
        k : int
    '''
    return current_rating + k*(n-1)*(actual_score-expected_score)

In [ ]:
player_ratings = {player: {1: {'date':(min(pd.to_datetime(df.date)) - pd.Timedelta(days=1)),
                               'rating':1000}} for player in df.player.unique()}
current_ratings = {player: 1000 for player in df.player.unique()}
i = 2

for game_id in unique_sorted_game_ids:
    game_df = df[df['game_id'] == game_id]
    
    new_ratings = {}
    current_ratings_active_players = {player: rating for player, rating in current_ratings.items() if player in game_df.player.unique()}

    for player, place, date in zip(game_df.player, game_df.place, game_df.date):
        expected = expected_score(s=current_ratings_active_players, player=player, n=game_df.shape[0])
        observed = exp_score(p=place, n=game_df.shape[0])
        updated =  update_rating(current_rating=current_ratings_active_players[player], 
                                 expected_score=expected, 
                                 actual_score=observed, 
                                 n=game_df.shape[0])
        new_ratings[player] = {'previous': current_ratings_active_players[player],
                               'expected': expected,
                               'observed': observed,
                               'updated_rating': updated}

    for player in current_ratings.keys():
            if player in new_ratings.keys(): # player played in current game
                current_ratings[player] = new_ratings[player]['updated_rating']
                player_ratings[player][i] = {'date': pd.to_datetime(date), 'rating': new_ratings[player]['updated_rating']}
            else: # player did not play in current game
                player_ratings[player][i] = {'date': pd.to_datetime(date), 'rating': current_ratings[player]}
    
    i += 1
    
player_rating_dfs_list = [pd.DataFrame(player_ratings[player]).T.reset_index().assign(player=player)\
                          for player in player_ratings.keys()]
player_ratings_df = pd.concat(player_rating_dfs_list)
player_ratings_df.columns = ['game_number','date','rating','player']


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))

for player in ['Ben','Ezra','Matt','Pat','Tony']:#player_ratings_df.player.unique():
    player_rating_df = player_ratings_df[player_ratings_df['player']==player]
    ax.plot(player_rating_df.date, player_rating_df.rating, label=player)
ax.legend()
ax.set_title('Player Ratings');

# Corporation ELO

In [ ]:
corp_ratings = {corp: {1: {'date':(min(pd.to_datetime(df.date)) - pd.Timedelta(days=1)),
                             'rating':1000}} for corp in df.corporation.unique()}
current_ratings = {corp: 1000 for corp in df.corporation.unique()}
i = 2

for game_id in unique_sorted_game_ids:
    game_df = df[df['game_id'] == game_id]
    game_df['place'] = game_df['total_points'].rank(ascending=False)
    
    new_ratings = {}
    current_ratings_active_corps = {corp: rating for corp, rating in current_ratings.items() if corp in game_df.corporation.unique()}

    for corp, place, date in zip(game_df.corporation, game_df.place, game_df.date):
        expected = expected_score(s=current_ratings_active_corps, player=corp, n=game_df.shape[0])
        observed = linear_score(p=place, n=game_df.shape[0])
        updated =  update_rating(current_rating=current_ratings_active_corps[corp], 
                                 expected_score=expected, 
                                 actual_score=observed, 
                                 n=game_df.shape[0])
        new_ratings[corp] = {'previous': current_ratings_active_corps[corp],
                               'expected': expected,
                               'observed': observed,
                               'updated_rating': updated}

    for corp in current_ratings.keys():
            if corp in new_ratings.keys(): # corp played in current game
                current_ratings[corp] = new_ratings[corp]['updated_rating']
                corp_ratings[corp][i] = {'date': pd.to_datetime(date), 'rating': new_ratings[corp]['updated_rating']}
            else: # corp did not play in current game
                corp_ratings[corp][i] = {'date': pd.to_datetime(date), 'rating': current_ratings[corp]}
    
    i += 1
    
corp_rating_dfs_list = [pd.DataFrame(corp_ratings[corp]).T.reset_index().assign(corporation=corp)\
                          for corp in corp_ratings.keys()]
corp_ratings_df = pd.concat(corp_rating_dfs_list)
corp_ratings_df.columns = ['game_number','date','rating','corporation']


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,15))

for corp in set(df[df.corporation_origin=='Base'].corporation.unique()) - set(['Beginner']):
    corp_rating_df = corp_ratings_df[corp_ratings_df['corporation']==corp][1:100]
    ax.plot(corp_rating_df.date, corp_rating_df.rating, label=corp)
ax.legend()
ax.set_title('Corporation Ratings');